## Importing necessary modules

In [1]:
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)



## Word embeddings 

### Read the dataset into a list, providing logging information

In [2]:


data_garmin_df = pd.read_csv('data/Garmin_Connect.csv')
data_samsung_df = pd.read_csv('data/Samsung_Health.csv')
data_huawei_df = pd.read_csv('data/Huawei_Health.csv')

data = pd.concat([data_garmin_df, data_samsung_df, data_huawei_df], ignore_index=True)

print(data.head())

                                                                                                  data  \
0  Contrairement aux idées reçues le traceur GPS est très précis, dumoins avec vivoactive 3 Music.....   
1                                                                 Application très pratique et fiable.   
2                                                                                     jadore ma montre   
3                                Super application, je l'utilise synchronisé avec ma fenix3 et j'adore   
4                                                                                              Super !   

   score  rating  bug_report  feature_request  user_experience  
0      5       1           0                0                1  
1      5       1           0                0                0  
2      5       1           0                0                0  
3      5       1           0                0                1  
4      5       1           0          

## Clean the text using gensim's pre built cleaner

In [7]:
# Clean data using the built in cleaner in gensim
data['text_clean'] = data['data'].apply(lambda x: gensim.utils.simple_preprocess(x))
# create a label column for all the possible combinations of labels 'rating', 'bug_report', 'feature_request', 'user_experience'
data['label'] = data['rating'].astype(str)  + data['bug_report'].astype(str) + data['feature_request'].astype(str) + data['user_experience'].astype(str)

data.head()

,data,score,rating,bug_report,feature_request,user_experience,text_clean,label
0,"Contrairement aux idées reçues le traceur GPS est très précis, dumoins avec vivoactive 3 Music.....",5,1,0,0,1,"[contrairement, aux, idées, reçues, le, traceur, gps, est, très, précis, dumoins, avec, vivoacti...",1001
1,Application très pratique et fiable.,5,1,0,0,0,"[application, très, pratique, et, fiable]",1000
2,jadore ma montre,5,1,0,0,0,"[jadore, ma, montre]",1000
3,"Super application, je l'utilise synchronisé avec ma fenix3 et j'adore",5,1,0,0,1,"[super, application, je, utilise, synchronisé, avec, ma, fenix, et, adore]",1001
4,Super !,5,1,0,0,0,[super],1000


## Split data into train and test sets


In [30]:
# Encoding the label column
data['label']=data['label'].map(lambda x: int(x))
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['text_clean'], data['label'] , test_size=0.2,stratify=data['label'], random_state=42)

## Train the Word2Vec model

In [31]:
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)
#w2v_model.wv.index_to_key

We can use our trained Word2Vec model to find similarities between words. Let's find the most similar words for 'compatibilité'

In [32]:
# Find the most similar words to "compatibilité" based on word vectors from our trained model
w2v_model.wv.most_similar('compatibilité')

[('donc', 0.9950855374336243),
 ('tel', 0.9950345754623413),
 ('avis', 0.9950264096260071),
 ('écran', 0.9949817061424255),
 ('fit', 0.9949743151664734),
 ('trouver', 0.9949272871017456),
 ('pendant', 0.9949046969413757),
 ('voir', 0.9949004054069519),
 ('pouvoir', 0.9948643445968628),
 ('vais', 0.9948185682296753)]

### Generate aggregated sentence vectors based on the word vectors for each word in the sentences

In [33]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

C:\Users\radio\AppData\Local\Temp\ipykernel_6392\22094319.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
C:\Users\radio\AppData\Local\Temp\ipykernel_6392\22094319.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


### Problem: inconsistent number of features in each sentence

In [34]:
# Why is the length of the sentence different than the length of the sentence vector?
for i, v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]), len(v))

26 26
7 7
15 14
1 1
17 16
78 73
13 12
28 28
13 13
14 14
17 16
19 19
1 1
2 0
3 3
20 20
30 28
8 8
36 34
27 27
58 58
75 72
60 56
1 1
10 9
27 27
46 45
6 6
26 26
6 6
5 5
1 1
6 6
36 32
11 11
13 13
2 1
15 15
18 18
1 1
74 72
50 48
15 15
1 1
1 1
11 11
8 8
46 45
7 6
16 15
7 7
1 1
5 5
6 6
40 40
20 20
14 13
22 21
0 0
3 3
6 6
16 15
15 13
7 7
5 5
15 15
46 44
1 1
4 4
2 2
15 15
4 4
18 18
25 25
22 22
19 17
12 12
2 2
4 4
71 69
39 38
12 11
48 46
5 5
17 17
4 4
16 14
87 84
2 2
66 66
50 46
6 5
54 53
8 8
14 13
9 9
16 16
22 22
14 14
79 78
1 1
1 1
24 24
5 4
49 48
10 9
11 11
2 2
11 11
16 16
6 4
6 6
1 1
9 7
22 21
6 6
8 8
8 7
6 6
11 11
6 6
5 5
27 26
2 2
23 20
20 19
3 3
37 36
15 15
3 3
15 14
13 13
64 60
2 2
15 15
49 48
17 17
29 29
2 2
2 2
12 12
81 78
80 78
4 4
7 4
3 2
2 2
12 12
1 1
2 2
34 34
15 15
1 1
1 1
7 7
21 17
15 15
37 36
9 9
1 1
5 5
1 1
43 42
5 5
1 1
65 62
2 2
23 22
8 8
5 5
1 1
4 4
6 6
2 2
25 25
37 36
17 17
19 15
62 61
1 0
82 78
7 7
47 47
25 23
1 1
6 6
18 18
45 41
14 14
10 10
28 28
2 1
14 14
8 8
66 59
14 14


If we were to pass these values into a machine learning algorithm, it would throw an error, because of the inconsistent amount of features in each data entry. In order to combat this, we are going to take an element wise average: We saw that there are two word vectors for the first sentence. Each of those word vectors is of size 100 because that’s the way we set it when we trained our model. What we’re going to do is we’re going to average the first element across those four word vectors and store that as the first entry in our final vector. Then we’ll do the same thing for the second element, and for the third, and so on. What we’ll end up with is now a single vector of length 100 that represents each text by averaging those word vectors for the words that were represented in that text message.

In [35]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

X_train_vect_avg

[array([-0.11723045,  0.33190727, -0.03710575,  0.01051122,  0.02966468,
        -0.7259443 ,  0.3821966 ,  0.9159901 , -0.506388  , -0.36348325,
        -0.15903777, -0.5561715 , -0.02135096,  0.14325462,  0.08693056,
        -0.304309  ,  0.2886495 , -0.12542675, -0.1216257 , -1.0006987 ,
         0.21185745, -0.09117541,  0.10411664, -0.2303215 , -0.19247085,
        -0.09304903, -0.30261597, -0.2042081 , -0.28014857,  0.10032485,
         0.433375  ,  0.0299611 ,  0.2742999 , -0.4115057 , -0.18915445,
         0.51754016,  0.28321883, -0.18813016, -0.09335552, -0.71648365,
         0.12104842, -0.55631435, -0.16504884,  0.07460132,  0.21873324,
        -0.35355222, -0.23546958, -0.05131832,  0.3168531 ,  0.23208226,
         0.24762453, -0.46961597,  0.06053131, -0.06400467, -0.2848507 ,
         0.24608675,  0.18197665, -0.04233652, -0.27691463,  0.2525545 ,
        -0.0529419 ,  0.01442839,  0.19961044,  0.00716587, -0.660129  ,
         0.40855077,  0.29017833,  0.5941886 , -0.6

Checking that our sentence word vectors are of consistent 100 length:

In [36]:

for i, v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]), len(v))

26 100
7 100
15 100
1 100
17 100
78 100
13 100
28 100
13 100
14 100
17 100
19 100
1 100
2 100
3 100
20 100
30 100
8 100
36 100
27 100
58 100
75 100
60 100
1 100
10 100
27 100
46 100
6 100
26 100
6 100
5 100
1 100
6 100
36 100
11 100
13 100
2 100
15 100
18 100
1 100
74 100
50 100
15 100
1 100
1 100
11 100
8 100
46 100
7 100
16 100
7 100
1 100
5 100
6 100
40 100
20 100
14 100
22 100
0 100
3 100
6 100
16 100
15 100
7 100
5 100
15 100
46 100
1 100
4 100
2 100
15 100
4 100
18 100
25 100
22 100
19 100
12 100
2 100
4 100
71 100
39 100
12 100
48 100
5 100
17 100
4 100
16 100
87 100
2 100
66 100
50 100
6 100
54 100
8 100
14 100
9 100
16 100
22 100
14 100
79 100
1 100
1 100
24 100
5 100
49 100
10 100
11 100
2 100
11 100
16 100
6 100
6 100
1 100
9 100
22 100
6 100
8 100
8 100
6 100
11 100
6 100
5 100
27 100
2 100
23 100
20 100
3 100
37 100
15 100
3 100
15 100
13 100
64 100
2 100
15 100
49 100
17 100
29 100
2 100
2 100
12 100
81 100
80 100
4 100
7 100
3 100
2 100
12 100
1 100
2 100
34 100
15 100
1

### Import the RandomForestClassifier and fit it with the averaged word vectors

In [37]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [38]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)
print(len(y_pred))

1200


In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, y_pred, average='macro', zero_division='warn')
recall = recall_score(y_test, y_pred, average='macro')
f1_score_var = f1_score(y_test, y_pred, average='macro')
print('Precision: {} / Recall: {} / Accuracy: {}, F1-score: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3), round(f1_score_var, 3)))

Precision: 0.221 / Recall: 0.189 / Accuracy: 0.577, F1-score: 0.186


c:\Python38\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
